In [1]:
import requests
import pandas as pd
import os
import json
from datetime import datetime
import mysql.connector

# Requesting assets_id list from MYSQL

In [2]:
mysql_password = os.getenv('MYSQL_PASSWORD')

mysql_connection = mysql.connector.connect(
  host = "localhost",
  user = "root",
  password = mysql_password,
  database = 'cripto_infos'
)

In [3]:
cursor = mysql_connection.cursor()

In [4]:
query_select_assets = "SELECT asset_id FROM assets"

In [5]:
cursor.execute(query_select_assets)
assets_tuple_list = cursor.fetchall()

In [6]:
asset_list = []
for asset_tuple in assets_tuple_list:
    asset = asset_tuple[0]
    asset_list.append(asset)

In [8]:
cursor.close()

True

# COLLECTING DATA

In [9]:
def find_assets_info(assets_list, api_key, url):
    
    json_list = []

    for asset in assets_list:

        # making a url for each asset
        new_url = url + asset

        # requesting data
        headers = {
            "Authorization": api_key,
            "Content-Type": "application/json"
        }
        response = requests.get(url=new_url, headers=headers)

        # loading json
        response_json = json.loads(response.text)

        # spliting json data
        main_data = response_json['data']
        time_stamp = response_json['timestamp'] / 1000.0 # dividing by 1000 because it's in microseconds

        # putting the timestamp as datetime inside the main json
        main_data['date'] = datetime.fromtimestamp(time_stamp)

        # adding the json into a list
        json_list.append(main_data)
    
    return json_list

In [10]:
api_key = os.getenv('COINCAP')
url = 'https://api.coincap.io/v2/assets/'

json_list = find_assets_info(asset_list, api_key, url)

In [11]:
df = pd.DataFrame(json_list)

In [12]:
df = df.drop(columns=['symbol', 'name', 'maxSupply', 'marketCapUsd', 'vwap24Hr', 'explorer'])

In [13]:
# taking away the time of date column
df['date'] = pd.to_datetime(df['date']).dt.date

In [14]:
# fixing data types
df['rank'] = df['rank'].astype(int)
df['supply'] = df['supply'].astype(float)
df['volumeUsd24Hr'] = df['volumeUsd24Hr'].astype(float)
df['priceUsd'] = df['priceUsd'].astype(float)
df['changePercent24Hr'] = df['changePercent24Hr'].astype(float)

In [16]:
new_column_names = {'id':'asset_id', 'rank':'rank_of_day', 'volumeUsd24Hr':'volume_trading_usd', 'priceUsd':'price', 'changePercent24Hr':'per_price_change', 'date':'date_d'}

In [17]:
df = df.rename(columns=new_column_names)

# LOADING DATA INTO MYSQL

In [19]:
cursor = mysql_connection.cursor()

In [22]:
for index, row in df.iterrows():
    data = (row['asset_id'], row['rank_of_day'], row['supply'], row['volume_trading_usd'], row['price'], row['per_price_change'], row['date_d'])

    query = '''
    INSERT infos (asset_id, rank_of_day, supply, volume_trading_usd, price, per_price_change, date_d)

    VALUES (%s, %s, %s, %s, %s, %s, %s)'''

    cursor.execute(query, data)

In [23]:
mysql_connection.commit()
mysql_connection.close()